In [13]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

### Choose a time partition

In [73]:
tp = 4

Load Database

In [74]:
data  = pd.read_csv(f'datos\counted\d{tp}_counted.csv', index_col=0)

In [75]:
data.columns

Index(['avisoid', 'empresaid', 'avisofechapublicacion', 'avisovacante',
       'mostrarsueldo', 'avisoexperiencia', 'expiracion', 'dias',
       'avisorepublicacion', 'avisocargo', 'areanombre',
       'actividadempresanombre', 'avisocuerpo', 'disponibilidadnombre',
       'avisoduracioncont', 'avisolugartrabajo', 'gradoescolarnombre',
       'situacionestudios', 'avisorequisitos', 'carreras', 'estado',
       'endpagado_o_gratuito', 'nivelnombre', 'carreras_array',
       'sueldoestimado', '_merge', 'teletrabajo_', 'avcu_b1', 'avca_b1',
       'avre_b1', 'avcu_b2', 'avca_b2', 'avre_b2', 'avcu_b3', 'avca_b3',
       'avre_b3', 'avcu_b4', 'avca_b4', 'avre_b4', 'bucket1', 'bucket2',
       'bucket3', 'bucket4', 'wfh', 'duplicates_avisocuerpo',
       'duplicates_avisocargo', 'similars', 'pseudoindex'],
      dtype='object')

In [76]:
data['avisofechapublicacion'] = pd.to_datetime(data['avisofechapublicacion'])

### Choose a window size in days

In [77]:
# Calculate the start and end dates
start_date = data['avisofechapublicacion'].min()
end_date = data['avisofechapublicacion'].max()

# Create a date range with one-day intervals
date_range = pd.date_range(start=start_date, end=end_date, freq='3M') #3M stands for three months

# Create a Pandas Series with the date range
dates = pd.Series(date_range)


avisocuerpo, empresaid, avisolugartrabajo

In [79]:
data['duplicates_avisocuerpo'] = False

In [80]:
for x in range(len(dates)-1):
    start_date = dates[x]
    final_date = dates[x+1]

    # Sub set with the date window
    newData = data[(data['avisofechapublicacion'] >= start_date) & (data['avisofechapublicacion'] < final_date)]

    # Another Data frame with the duplicates for the newData subset
    duplicates = newData[newData.duplicated(subset=['empresaid', 'avisocuerpo', 'avisolugartrabajo'], keep='first')]  # Dont consider the first realiztion, mark everything else as duplicate

    # Check wich ad from Data dataFrame is in the duplicates dataFrame
    data['duplicates_avisocuerpo'] = data['duplicates_avisocuerpo'] + data.index.isin(duplicates.index)


In [82]:
data['duplicates_avisocuerpo'].value_counts()

False    575144
True     155167
Name: duplicates_avisocuerpo, dtype: int64

avisocargo, avisocuerpo, empresaid, avisolugartrabajo

In [83]:
data['duplicates_avisocargo'] = False

In [84]:
for x in range(len(dates)-1):
    start_date = dates[x] 
    final_date = dates[x+1]

    # Sub set with the date window
    newData = data[(data['avisofechapublicacion'] >= start_date) & (data['avisofechapublicacion'] < final_date)]

    # Another Data frame with the duplicates for the newData subset
    duplicates = newData[newData.duplicated(subset=['empresaid', 'avisocargo', 'avisocuerpo', 'avisolugartrabajo'], keep='first')]  # Dont consider the first realiztion, mark everything else as duplicate

    # Check wich ad from Data dataFrame is in the duplicates dataFrame
    data['duplicates_avisocargo'] = data['duplicates_avisocargo'] + data.index.isin(duplicates.index)

In [85]:
data['duplicates_avisocargo'].value_counts()

False    615376
True     114935
Name: duplicates_avisocargo, dtype: int64

Save the data

In [86]:
data.to_csv(f'datos\counted\d{tp}_counted.csv')